In [2]:
import os
from glob import glob

import netCDF4 as nc
import xarray as xr
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt

In [6]:
level2_source = '/g/data/rq0/level_2'
field = 'SHI'
radar_id = 69
request_lat = -30.615571 #NE Tablelands solar farm
request_lon = 151.602352
request_radius = 10 #km
threshold = 30

min_lat = request_lat-0.1
max_lat = request_lat+0.1
min_lon = request_lon-0.1
max_lon = request_lon+0.1


In [7]:
#list all level 2 files
level_2_ffn_list = sorted(glob(f'{level2_source}/{radar_id}/{field}/*.nc'))

#load coords
with nc.Dataset(level_2_ffn_list[0]) as ds:
    lat = ds.variables['latitude'][:]
    lon = ds.variables['longitude'][:]
    x = ds.variables['x'][:] / 1000 #convert to km
    y = ds.variables['y'][:] / 1000 # convert to km
    x_grid, y_grid = np.meshgrid(x, y)

#calculate grid location closest to request
dist_deg = np.sqrt((lat-request_lat)**2 + (lon-request_lon)**2)
closest_index_array = np.where(dist_deg == np.min(dist_deg))
closest_index = (closest_index_array[0][0], closest_index_array[1][0])

#calculate all points within a radius of the closest point
dist_km = np.sqrt((x_grid-x_grid[closest_index])**2 + (y_grid-y_grid[closest_index])**2)
radius_outside = dist_km > request_radius
radius_inside = ~radius_outside

x_min = np.min(x_grid[radius_inside])*1000
x_max = np.max(x_grid[radius_inside])*1000
y_min = np.min(y_grid[radius_inside])*1000
y_max = np.max(y_grid[radius_inside])*1000
print('x min', x_min)
print('x max', x_max)
print('y min', y_min)
print('y max', y_max)




x min 125000.0
x max 145000.0
y min 34000.0
y max 54000.0


In [8]:
#load daily data
hailstorm_list = []
for level_2_ffn in level_2_ffn_list:
    with xr.open_dataset(level_2_ffn) as ds:
        cropped_ds = ds.sel(x=slice(x_min, x_max), y=slice(y_min, y_max))
        shi = cropped_ds['shi'].data[:]
        mesh = 15.096 * shi**0.206
        if np.sum(mesh>threshold) == 0:
            continue
        else:
            date = datetime.strptime(os.path.basename(level_2_ffn)[3:11], '%Y%m%d')
            hailstorm_list.append([date, np.nanmax(mesh)])
            print([date, np.nanmax(mesh)])

[datetime.datetime(2010, 12, 13, 0, 0), 41.11806]
[datetime.datetime(2010, 12, 14, 0, 0), 37.006176]
[datetime.datetime(2010, 12, 16, 0, 0), 44.918537]
[datetime.datetime(2011, 1, 3, 0, 0), 47.055843]
[datetime.datetime(2011, 1, 19, 0, 0), 48.271687]
[datetime.datetime(2011, 2, 7, 0, 0), 31.130627]
[datetime.datetime(2011, 2, 13, 0, 0), 32.572487]
[datetime.datetime(2011, 4, 2, 0, 0), 35.706814]
[datetime.datetime(2011, 9, 24, 0, 0), 32.770744]
[datetime.datetime(2011, 9, 30, 0, 0), 34.13616]
[datetime.datetime(2011, 10, 25, 0, 0), 38.68642]
[datetime.datetime(2011, 10, 30, 0, 0), 30.701397]
[datetime.datetime(2011, 11, 29, 0, 0), 44.102917]
[datetime.datetime(2012, 1, 5, 0, 0), 31.40074]
[datetime.datetime(2012, 2, 8, 0, 0), 35.384506]
[datetime.datetime(2012, 9, 17, 0, 0), 35.79787]
[datetime.datetime(2012, 11, 25, 0, 0), 31.893837]
[datetime.datetime(2012, 12, 2, 0, 0), 37.140163]
[datetime.datetime(2012, 12, 23, 0, 0), 31.560875]
[datetime.datetime(2012, 12, 25, 0, 0), 36.65005]
[d

In [12]:
#write to csv
import csv

with open('radar_69_NE_solar_haildays.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in hailstorm_list:
        writer.writerow([item[0].strftime('%Y-%m-%d')])
